In [ ]:
import os, shutil, sys
from google.colab import files

# --- 1️⃣ Define cleanup helper ---
def force_delete(folder):
    if os.path.exists(folder):
        try:
            shutil.rmtree(folder)
            print(f"✅ Removed old folder: {folder}")
        except Exception as e:
            print(f"⚠️ Normal delete failed for {folder}: {e}")
            # Try using shell command as fallback
            os.system(f"rm -rf {folder}")
            print(f"💪 Force-removed with shell: {folder}")

# --- 2️⃣ Clean up both possible folders ---
for f in ["nlp_project", "updated_nlp_project"]:
    # Unload from sys.modules to unlock files
    for mod in list(sys.modules.keys()):
        if f in mod:
            del sys.modules[mod]
    force_delete(f)

In [ ]:


# --- 3️⃣ Upload ZIP ---
uploaded = files.upload()

# --- 4️⃣ Unzip directly into root directory ---
!unzip -q updated_nlp_project.zip -d .

# --- 5️⃣ CD into correct folder ---
if os.path.exists("updated_nlp_project/nlp_project"):
    %cd updated_nlp_project/nlp_project
elif os.path.exists("nlp_project"):
    %cd nlp_project
else:
    print("⚠️ Could not find nlp_project folder after unzipping!")

print("✅ Now in:", os.getcwd())


Saving updated_nlp_project.zip to updated_nlp_project.zip
/content/updated_nlp_project/nlp_project
✅ Now in: /content/updated_nlp_project/nlp_project


In [ ]:
!pip install torch numpy matplotlib scikit-learn seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 146.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 149.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [ ]:
!python generate_data.py

[array(['yellow', 'cross', '4'], dtype='<U6'), array(['blue', 'star', '1'], dtype='<U6'), array(['green', 'cross', '3'], dtype='<U6'), array(['yellow', 'cross', '2'], dtype='<U6'), array(['blue', 'square', '3'], dtype='<U6'), 'SEP', 'C3', 'EOS', array(['green', 'circle', '1'], dtype='<U6'), 'SEP', 'C2']
[array(['blue', 'cross', '1'], dtype='<U6'), array(['green', 'star', '3'], dtype='<U6'), array(['blue', 'star', '4'], dtype='<U6'), array(['red', 'cross', '2'], dtype='<U6'), array(['green', 'square', '1'], dtype='<U6'), 'SEP', 'C1', 'EOS', array(['red', 'circle', '3'], dtype='<U6'), 'SEP', 'C2']
[array(['green', 'cross', '3'], dtype='<U6'), array(['red', 'circle', '2'], dtype='<U6'), array(['green', 'circle', '4'], dtype='<U6'), array(['red', 'square', '1'], dtype='<U6'), array(['yellow', 'star', '3'], dtype='<U6'), 'SEP', 'C1', 'EOS', array(['blue', 'star', '3'], dtype='<U6'), 'SEP', 'C1']
[array(['blue', 'star', '3'], dtype='<U6'), array(['yellow', 'circle', '2'], dtype='<U6'), array

In [ ]:
import os

def load_sequences(filename):
    """Load lines as a stripped set."""
    with open(filename, 'r') as f:
        return set(line.strip() for line in f if line.strip())

def save_sequences(filename, seqs):
    """Save cleaned unique sequences to file."""
    with open(filename, 'w') as f:
        for seq in sorted(seqs):
            f.write(seq + '\n')

def remove_overlaps(train_file, val_file, test_file):
    # --- Load ---
    train = load_sequences(train_file)
    val   = load_sequences(val_file)
    test  = load_sequences(test_file)

    print("Before cleanup:")
    print(f"  Train: {len(train)}")
    print(f"  Val:   {len(val)}")
    print(f"  Test:  {len(test)}")

    # --- Detect overlaps ---
    train_val_overlap = train & val
    train_test_overlap = train & test
    val_test_overlap = val & test

    total_overlaps = len(train_val_overlap | train_test_overlap | val_test_overlap)
    if total_overlaps == 0:
        print("✅ No overlaps found.")
        return

    print(f"⚠️ Detected {total_overlaps} overlapping sequences.")
    print(f"    Train ∩ Val:  {len(train_val_overlap)}")
    print(f"    Train ∩ Test: {len(train_test_overlap)}")
    print(f"    Val ∩ Test:   {len(val_test_overlap)}")

    # --- Clean strategy: keep priority order Train > Val > Test ---
    # Remove from val anything already in train
    val -= train
    # Remove from test anything already in train or val
    test -= (train | val)

    # --- Save cleaned sets ---
    save_sequences(train_file, train)
    save_sequences(val_file, val)
    save_sequences(test_file, test)

    # --- Report ---
    print("\nAfter cleanup:")
    print(f"  Train: {len(train)} sequences")
    print(f"  Val:   {len(val)} sequences")
    print(f"  Test:  {len(test)} sequences")
    print("✅ All splits are now unique and leakage-free.")

# --- Run the cleaner ---
remove_overlaps("train_data.txt", "validation_data.txt", "test_data.txt")


Before cleanup:
  Train: 49993
  Val:   2000
  Test:  5000
⚠️ Detected 1 overlapping sequences.
    Train ∩ Val:  1
    Train ∩ Test: 0
    Val ∩ Test:   0

After cleanup:
  Train: 49993 sequences
  Val:   1999 sequences
  Test:  5000 sequences
✅ All splits are now unique and leakage-free.


In [ ]:
# Load sequences
with open("train_data.txt") as f:
    train_lines = set(line.strip() for line in f)

with open("test_data.txt") as f:
    test_lines = set(line.strip() for line in f)
with open("validation_data.txt") as f:
    val_lines = set(line.strip() for line in f)

# Find overlapping sequences
overlap = train_lines & test_lines & val_lines
print(f"Number of overlapping sequences: {len(overlap)}")
print("Overlapping sequence(s):")
for seq in overlap:
    print(seq)


Number of overlapping sequences: 0
Overlapping sequence(s):


In [ ]:
!python fix_leakage.py

Loading datasets...
Original counts -> Train: 49993, Val: 1999, Test: 5000
Filtered counts -> Val: 1975, Test: 4938
✅ Saved cleaned validation to validation_clean.txt and test to test_clean.txt


In [ ]:
!python leakage_checker.py

Loading datasets...
Train: 49993, Val: 1975, Test: 4938

Partial sequence overlap (first 4 tokens):
  Train ∩ Val: 0 sequences
  Train ∩ Test: 0 sequences

Label leakage (target after SEP appears in input):
  Val: 0 sequences
  Test: 0 sequences

Feature leakage (target appears anywhere in input):
  Val: 0 sequences
  Test: 0 sequences

✅ Leakage check complete.


In [ ]:
!python train.py

Loading data...
Sanity check (validation batch): unique tokens at targets[:, -1]:
  token 64: 18 samples
  token 65: 12 samples
  token 66: 15 samples
  token 67: 19 samples
Using device: cuda
Tied output_layer.weight to embedding.token_embedding.weight
Starting training...
Initial grad norms (sample): [3.953641414642334, 38.10582733154297, 37.51200485229492, 14.680550575256348, 13.833096504211426, 0.30161866545677185, 12.374910354614258, 1.6089750528335571]
Epoch [1/40], Batch [0/782], Loss(full+aux): 191.1141
Epoch [1/40], Batch [100/782], Loss(full+aux): 4.9499
Epoch [1/40], Batch [200/782], Loss(full+aux): 4.7170
Epoch [1/40], Batch [300/782], Loss(full+aux): 4.3445
Epoch [1/40], Batch [400/782], Loss(full+aux): 4.0276
Epoch [1/40], Batch [500/782], Loss(full+aux): 3.8675
Epoch [1/40], Batch [600/782], Loss(full+aux): 3.8120
Epoch [1/40], Batch [700/782], Loss(full+aux): 3.8933
Epoch [1/40]
  Train Loss (full+aux): 5.3543
  Validation Loss (full-seq CE): 3.6028
  Validation Loss (c

In [ ]:
# Quick data leakage check
def load_sequences(fname):
    with open(fname, 'r') as f:
        return set(line.strip() for line in f)

train_seqs = load_sequences("train_data.txt")
val_seqs   = load_sequences("validation_data.txt")
test_seqs  = load_sequences("test_data.txt")

print(f"Train: {len(train_seqs)} sequences")
print(f"Val:   {len(val_seqs)} sequences")
print(f"Test:  {len(test_seqs)} sequences")

# Check overlaps
train_val_overlap = train_seqs & val_seqs
train_test_overlap = train_seqs & test_seqs
val_test_overlap   = val_seqs & test_seqs

print(f"Train ∩ Val overlap: {len(train_val_overlap)} sequences")
print(f"Train ∩ Test overlap: {len(train_test_overlap)} sequences")
print(f"Val ∩ Test overlap: {len(val_test_overlap)} sequences")

if train_val_overlap or train_test_overlap or val_test_overlap:
    print("⚠️ Possible data leakage detected!")
else:
    print("✅ No overlapping sequences found.")


Train: 49993 sequences
Val:   1999 sequences
Test:  5000 sequences
Train ∩ Val overlap: 0 sequences
Train ∩ Test overlap: 0 sequences
Val ∩ Test overlap: 0 sequences
✅ No overlapping sequences found.


In [ ]:
!python train_improved.py

Loading data...
Sanity check (validation batch): unique tokens at targets[:, -1]:
  token 64: 18 samples
  token 65: 12 samples
  token 66: 15 samples
  token 67: 19 samples
Using device: cuda
Starting training...
Epoch 1 batch 0 sample grad norms: [0.04109564796090126, 0.04100159555673599, 0.038971059024333954, 0.04564875364303589, 1.0215439796447754, 1.0481762886047363, 0.5895255208015442, 0.5554810166358948, 0.0059204427525401115, 0.3616199493408203]
Epoch [1/40], Batch [0/782], Loss(full+aux): 4.4659
Epoch [1/40], Batch [100/782], Loss(full+aux): 3.1877
Epoch [1/40], Batch [200/782], Loss(full+aux): 3.1268
Epoch [1/40], Batch [300/782], Loss(full+aux): 3.1450
Epoch [1/40], Batch [400/782], Loss(full+aux): 3.0199
Epoch [1/40], Batch [500/782], Loss(full+aux): 3.0394
Epoch [1/40], Batch [600/782], Loss(full+aux): 3.0301
Epoch [1/40], Batch [700/782], Loss(full+aux): 3.0551
Epoch [1/40]  Train Loss(full+aux): 3.1016  Val Loss(full): 2.9721  Val Loss(category-only): 1.3924  Val Acc(cat

In [ ]:
!pip install umap-learn minisom

  Using cached minisom-2.3.5.tar.gz (12 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=d12188f0ac2352d2fb67674eef47f2b5b54f606f085814f41289cd4a0fae4512
  Stored in directory: /root/.cache/pip/wheels/19/db/95/5e53bc2b88a328217fdf9f2886cafbe86b0df274f4b601f572
Successfully built minisom


In [ ]:
!python evaluation_2.py

2025-10-31 13:23:37.220714: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761917017.240206   36832 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761917017.246196   36832 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-31 13:23:37.265462: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
🔬 Comparing Baseline vs Feature-based models
Loaded checkpoint transformer_wcst_best.pth
✅ Saved plot: evaluation_plo

In [ ]:
%matplotlib inline
!python evaluation.py

In [ ]:
from IPython.display import Image, display, HTML
import os
import json

def display_section_header(title, level=1):
    """Display a formatted section header"""
    if level == 1:
        border = "="*70
        print(f"\n{border}")
        print(f"{title:^70}")
        print(f"{border}\n")
    elif level == 2:
        print(f"\n{'─'*70}")
        print(f"  {title}")
        print(f"{'─'*70}\n")
    else:
        print(f"\n  📊 {title}")

def display_plot_if_exists(path, title=""):
    """Display plot if file exists, otherwise show warning"""
    if os.path.exists(path):
        if title:
            print(f"    {title}")
        display(Image(filename=path))
        return True
    else:
        print(f"    ⚠️  Not found: {os.path.basename(path)}")
        return False

# Main display logic
display_section_header("WCST TRANSFORMER EVALUATION RESULTS", level=1)

models = ["Baseline", "Features"]

for model_name in models:
    display_section_header(f"MODEL: {model_name}", level=1)
    model_dir = f"evaluation_plots/{model_name}"

    # 1. Training Dynamics
    display_section_header("Training Dynamics", level=2)
    display_plot_if_exists(f"{model_dir}/training_curves.png", "Loss and Accuracy Curves")

    # 2. Embedding Analysis
    display_section_header("Embedding Analysis", level=2)
    display_plot_if_exists(f"{model_dir}/embeddings_pca.png", "PCA Visualization")

    # Display t-SNE, UMAP, SOM for each attribute
    attributes = ["color", "shape", "quantity"]
    viz_types = ["tsne", "umap", "som"]

    for attr in attributes:
        print(f"\n  🔹 {attr.capitalize()} Embeddings:")
        for viz in viz_types:
            plot_path = f"{model_dir}/emb_{attr}_{viz}.png"
            if os.path.exists(plot_path):
                print(f"      • {viz.upper()}")
                display(Image(filename=plot_path))

    # 3. Model Interpretation
    display_section_header("Model Interpretation", level=2)
    display_plot_if_exists(f"{model_dir}/ablation.png", "Ablation Study")
    display_plot_if_exists(f"{model_dir}/attention_layer_0.png", "Attention - Layer 0")
    display_plot_if_exists(f"{model_dir}/attention_layer_3.png", "Attention - Layer 3")

    # 4. Performance Analysis
    display_section_header("Performance Analysis", level=2)
    display_plot_if_exists(f"{model_dir}/confusion_matrix.png", "Confusion Matrix")
    display_plot_if_exists(f"{model_dir}/consistency_check.png", "Consistency Check")

# Comparison Summary
display_section_header("COMPARISON SUMMARY", level=1)

summary_path = "evaluation_plots/comparison_summary.json"
if os.path.exists(summary_path):
    with open(summary_path, 'r') as f:
        summary = json.load(f)

    # Create comparison table
    comparison_data = []
    for model_key in ['baseline', 'features']:
        if model_key in summary and summary[model_key]:
            data = summary[model_key]
            sil = data['emb_summary']['silhouette']
            comparison_data.append({
                'Model': data['name'],
                'Test Acc': f"{data['failure']['overall']:.1%}",
                'Color Sil': f"{sil['color']:+.3f}",
                'Shape Sil': f"{sil['shape']:+.3f}",
                'Qty Sil': f"{sil['quantity']:+.3f}",
            })

    # Display as formatted table
    if comparison_data:
        print("\n  Model Performance Comparison:")
        print("  " + "─"*65)
        headers = comparison_data[0].keys()
        print(f"  {'Model':<12} {'Test Acc':<10} {'Color Sil':<12} {'Shape Sil':<12} {'Qty Sil':<10}")
        print("  " + "─"*65)
        for row in comparison_data:
            print(f"  {row['Model']:<12} {row['Test Acc']:<10} {row['Color Sil']:<12} {row['Shape Sil']:<12} {row['Qty Sil']:<10}")
        print("  " + "─"*65)

print(f"\n{'='*70}")
print(f"{'✅ Evaluation Complete!':^70}")
print(f"{'='*70}\n")

In [ ]:
%cd ..

/content/updated_nlp_project


In [ ]:
!zip -r nlp_project.zip nlp_project/


  adding: nlp_project/ (stored 0%)
  adding: nlp_project/.ipynb_checkpoints/ (stored 0%)
  adding: nlp_project/train.py (deflated 70%)
  adding: nlp_project/wcst.py (deflated 75%)
  adding: nlp_project/__pycache__/ (stored 0%)
  adding: nlp_project/__pycache__/model.cpython-311.pyc (deflated 59%)
  adding: nlp_project/__pycache__/model_improved.cpython-311.pyc (deflated 58%)
  adding: nlp_project/__pycache__/visualizations.cpython-311.pyc (deflated 50%)
  adding: nlp_project/__pycache__/wcst.cpython-311.pyc (deflated 56%)
  adding: nlp_project/__pycache__/wcst.cpython-313.pyc (deflated 53%)
  adding: nlp_project/__pycache__/train.cpython-311.pyc (deflated 53%)
  adding: nlp_project/__pycache__/data_loader.cpython-311.pyc (deflated 46%)
  adding: nlp_project/fix_leakage.py (deflated 60%)
  adding: nlp_project/training_metrics_features.json (deflated 57%)
  adding: nlp_project/validation_clean.txt (deflated 70%)
  adding: nlp_project/test_data.txt (deflated 72%)
  adding: nlp_project/vis

In [ ]:
%cd nlp_project

/content/updated_nlp_project/nlp_project


In [ ]:
!python permutation.py

Loaded checkpoint: transformer_wcst_best.pth
Original test accuracy: 0.9177804779262859
Zeroed-inputs test accuracy: 0.0
Random-inputs test accuracy: 0.0074929121101660594
Shuffled-inputs test accuracy: 0.043337383556095586

Interpretation:
- If zero/random/shuffled accuracies ≈ original accuracy, likely leakage or dataset mapping bug.
- If they drop to ≈0.25 (chance), model is using proper input tokens (no trivial leakage).
